# Model setup figures and tables


In [ ]:
%load_ext dotenv
%dotenv
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
import matplotlib.offsetbox as moffsetbox

from cmcrameri import cm

plt.ioff()

from src.config import get_config, get_dask_cluster
from src.plotting import FIGURE_WIDTH_S, FIGURE_WIDTH_M, FIGURE_WIDTH_L
from src.analysis import apply_buffer_zone_to_array
from src.job_generation import read_namelist

config = get_config()
cluster, client = get_dask_cluster(config)

In [ ]:
plt.rcParams["font.family"] = config.plotting.font_family
plt.rcParams["font.serif"] = config.plotting.font_serif
plt.rcParams["font.size"] = config.plotting.font_size

## Simulation timeline

Create a Gantt chart of the simulation timelines.


In [ ]:
cols = ["Precursor", "Main simulations", "Daytime", "Nighttime"]
rows = ["Begin", "End", "Duration"]
timeline = pd.DataFrame(index=cols, columns=rows)
timeline.loc["Precursor", "Begin"] = pd.Timestamp("2018-03-30 03:00:00")
timeline.loc["Precursor", "End"] = pd.Timestamp("2018-04-01 04:00:00")
timeline.loc["Main simulations", "Begin"] = pd.Timestamp("2018-03-31 03:00:00")
timeline.loc["Main simulations", "End"] = pd.Timestamp("2018-04-01 04:00:00")
timeline.loc["Daytime", "Begin"] = pd.Timestamp("2018-03-31 12:00:00")
timeline.loc["Daytime", "End"] = pd.Timestamp("2018-03-31 16:00:00")
timeline.loc["Nighttime", "Begin"] = pd.Timestamp("2018-04-01 00:00:00")
timeline.loc["Nighttime", "End"] = pd.Timestamp("2018-04-01 04:00:00")
timeline.loc[:, "Duration"] = timeline.loc[:, "End"] - timeline.loc[:, "Begin"]

In [ ]:
timeline

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(FIGURE_WIDTH_S, 1.2), constrained_layout=True)
ax.barh(
    y=timeline.index,
    width=timeline["Duration"],
    left=timeline["Begin"],
    color="tab:orange",
    zorder=3,
)
ax.invert_yaxis()

In [ ]:
ax.set_xlim((pd.Timestamp("2018-03-30 00:00:00"), pd.Timestamp("2018-04-01 06:00:00")))
ax.xaxis.set_major_locator(mdates.HourLocator(interval=6))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H"))
ax.set_xlabel("Time (local/solar)")

ax.grid(axis="x", linewidth=0.5, color="tab:grey", zorder=0)

out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
fig.savefig(out_path / "model_timeline_gantt.pdf", dpi=300)
fig

## Precursor


### Surface description

Precursor surface is a randomized mix of vegetation patches, a table is sufficient to report the vegetation types.


In [ ]:
precursor_def_path = Path(config.path.data.jobs) / "slurb_pre_default"
precursor_def_static = xr.open_dataset(
    precursor_def_path / "INPUT" / "slurb_pre_default_static"
)

vegetation_type, vegetation_type_counts = np.unique(
    precursor_def_static["vegetation_type"], return_counts=True
)
vegetation_type_index = pd.Index(vegetation_type.astype(int), name="Vegetation type")
vegetation_type_fractions = vegetation_type_counts / vegetation_type_counts.sum()
pd.DataFrame(
    (vegetation_type_fractions * 100).round(1),
    index=vegetation_type_index,
    columns=["Fraction (%)"],
)

In [ ]:
print(
    pd.DataFrame(
        (vegetation_type_fractions * 100).round(1),
        index=vegetation_type_index,
        columns=["Fraction (%)"],
    ).to_latex(float_format="%.1f")
)

### Radiative forcing

Plot time series of radiative forcing.


In [ ]:
precursor_def_dynamic = xr.open_dataset(
    precursor_def_path / "INPUT" / "slurb_pre_default_dynamic"
)
baseline_p3d = read_namelist(
    Path(config.path.experiments.sensitivity) / "precursor_default_p3d.yml"
)
origin_date_time = pd.Timestamp(
    baseline_p3d["initialization_parameters"]["origin_date_time"]
)
precursor_def_dynamic["time_rad"] = (
    origin_date_time + pd.to_timedelta(precursor_def_dynamic["time_rad"].data, unit="s")
).values.astype("datetime64[ns]")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(FIGURE_WIDTH_L, 2.5), constrained_layout=True)
precursor_def_dynamic["rad_sw_in"].plot.line(
    x="time_rad", ax=ax, label="Incoming SW radiation"
)
precursor_def_dynamic["rad_lw_in"].plot.line(
    x="time_rad", ax=ax, label="Incoming LW radiation"
)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax.set_xlabel("Time (solar/local)")
ax.set_ylabel(r"Irradiance (W m$\mathdefault{^{-2}}$)")
ax.legend(loc="upper right")
out_path = Path(config.path.results) / "supplementary"
out_path.mkdir(parents=True, exist_ok=True)
fig.savefig(out_path / "precursor_radiation.pdf", dpi=300)
fig

## Dynamic input

A multi-panel plot showing applied forcing in the base case.


In [ ]:
precursor_def_pr = xr.open_mfdataset(
    str(precursor_def_path) + "/OUTPUT/slurb_pre_default_pr.00*.nc"
)
precursor_def_pr["time"] = pd.to_timedelta(precursor_def_pr["time"])
precursor_def_pr["time"] = (
    precursor_def_pr["time"]
    + pd.Timestamp("2018-03-30 03:00:00")
    - pd.Timedelta(minutes=15)
)
precursor_def_pr = precursor_def_pr.where(
    precursor_def_pr["time"] >= pd.Timestamp("2018-03-31 04:00:00"), drop=True
)
precursor_def_pr["v"] = (("time", "zu"), precursor_def_pr["v"].values)
precursor_def_pr["u_abs"] = np.sqrt(
    precursor_def_pr["u"] ** 2 + precursor_def_pr["v"] ** 2
)
precursor_def_pr["T"] = (
    precursor_def_pr["theta"] - (9.81 / 1000) * precursor_def_pr["ztheta"]
)

precursor_def_pr["wdir"] = (
    180 + np.arctan2(precursor_def_pr["u"], precursor_def_pr["v"]) * 180 / np.pi
) % 360
# precursor_def_pr["rh"] = precursor_def_pr["rh"]*100

In [ ]:
precursor_def_pr["u_abs"]

In [ ]:
fig = plt.figure()
fig, axs = plt.subplots(
    5,
    1,
    sharex=True,
    figsize=(FIGURE_WIDTH_S, 6),
    constrained_layout=True,
)

axs[-1].set_xlim(
    (pd.Timestamp("2018-03-31 04:00:00"), pd.Timestamp("2018-04-01 04:00:00"))
)

precursor_def_pr["u_abs"].plot.imshow(
    x="time",
    y="zu",
    interpolation="bicubic",
    vmin=0.0,
    cmap="viridis",
    ax=axs[0],
    cbar_kwargs={
        "fraction": 0.2,
        "aspect": 10,
        "pad": 0.0,
        "label": r"$U$ ($\mathrm{m}~\mathrm{s}^{-1}$)",
        "ticks": [0, 2, 4, 6],
    },
)
precursor_def_pr["wdir"].plot.imshow(
    x="time",
    y="zu",
    interpolation="bicubic",
    vmin=270 - 45,
    vmax=270 + 45,
    cmap="BrBG",
    ax=axs[1],
    cbar_kwargs={
        "fraction": 0.2,
        "aspect": 10,
        "pad": 0.0,
        "label": r"Wind direction (deg)",
        "extend": "neither",
        "ticks": [225, 270, 315],
    },
)
(precursor_def_pr["theta"] - 273.15).plot.imshow(
    x="time",
    y="ztheta",
    interpolation="bicubic",
    cmap=cm.lipari,
    vmin=8,
    vmax=18,
    ax=axs[2],
    cbar_kwargs={
        "fraction": 0.2,
        "aspect": 10,
        "pad": 0.0,
        "label": r"$\theta$ ($^\circ\mathrm{C}$)",
        "extend": "neither",
        "ticks": [8, 10, 12, 14, 16, 18],
    },
)
precursor_def_pr["rh"].plot.imshow(
    x="time",
    y="zrh",
    interpolation="bicubic",
    vmin=0,
    vmax=90,
    cmap=cm.lapaz_r,
    ax=axs[3],
    cbar_kwargs={
        "fraction": 0.2,
        "aspect": 10,
        "pad": 0.0,
        "label": r"RH (%)",
        "extend": "neither",
    },
)

precursor_def_dynamic["rad_sw_in"].plot.line(
    x="time_rad", ax=axs[4], label=r"$S^{\Downarrow}$"
)
precursor_def_dynamic["rad_lw_in"].plot.line(
    x="time_rad", ax=axs[4], label=r"$L^{\Downarrow}$"
)

for ax in axs[0:-1]:
    ax.set_ylim((0, 2000))
    ax.set_yticks((0, 1000, 2000), ("0 km", "1 km", "2 km"))
    ax.set_ylabel("$z$ (m)")
    ax.set_xlabel("")

axs[0].add_artist(
    moffsetbox.AnchoredText("(a)", loc="upper left", pad=0.2, borderpad=0)
)
axs[1].add_artist(
    moffsetbox.AnchoredText("(b)", loc="upper left", pad=0.2, borderpad=0)
)
axs[2].add_artist(
    moffsetbox.AnchoredText("(c)", loc="upper left", pad=0.2, borderpad=0)
)
axs[3].add_artist(
    moffsetbox.AnchoredText("(d)", loc="upper left", pad=0.2, borderpad=0)
)
axs[4].add_artist(
    moffsetbox.AnchoredText("(e)", loc="upper left", pad=0.2, borderpad=0)
)

axs[-1].set_ylim((0, None))
axs[-1].set_xlabel("Time (local/solar)")
axs[-1].xaxis.set_major_locator(mdates.HourLocator(interval=4))
axs[-1].xaxis.set_major_formatter(mdates.DateFormatter("%H"))

axs[-1].set_ylabel(r"Irradiance (W m$\mathdefault{^{-2}}$)")
axs[-1].legend(loc="upper right")

out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
fig.savefig(out_path / "forcing_base.pdf", dpi=300)
fig

Compute diurnal mean values of the forcing for the manuscript.


In [ ]:
rad_sw_mean = precursor_def_dynamic["rad_sw_in"].mean()
rad_lw_mean = precursor_def_dynamic["rad_lw_in"].mean()

In [ ]:
print(
    f"SW rad diurnal mean and modification: {rad_sw_mean:.0f} ± {0.5*rad_sw_mean:.0f}"
)
print(f"LW rad diurnal mean and modification: {rad_lw_mean:.0f} ± {10.:.0f}")

## Sensitivity tests


### Domain setup


In [ ]:
resolution = (16, 16)
gridpoints = (256, 256)
urban_offset = (2112 + 2 * 896 - 1536, 1280)

In [ ]:
vegetation_map = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "INPUT"
    / "slurb_pre_default_static"
)["vegetation_type"]
vegetation_coarse_classification = xr.apply_ufunc(
    lambda x: np.where(np.isin(x, [1, 2, 3, 8, 9, 10, 11, 12, 13, 14, 15, 16]), 1, 2),
    vegetation_map,
)

In [ ]:
plt.ioff()
fig, axs = plt.subplots(
    2,
    1,
    height_ratios=[1, 10],
    figsize=(FIGURE_WIDTH_S, 9 / 2.56),
    layout="constrained",
)
ax = axs[1]
legend_ax = axs[0]
legend_ax.set_axis_off()
ax.set_aspect("equal")
ax.set_xlim((0.0, resolution[0] * gridpoints[0]))
ax.set_ylim((0.0, resolution[1] * gridpoints[1]))
ax.set_xticks(np.arange(0, 4001, 1000))
ax.set_yticks(np.arange(0, 4001, 1000))
ax.set_xlabel(r"$x$ (m)")
ax.set_ylabel(r"$y$ (m)")

In [ ]:
ax.text(
    168,
    resolution[1] * gridpoints[1] // 2,
    r"$\downarrow$ Inflow boundary $\downarrow$",
    va="center",
    ha="center",
    rotation="vertical",
    fontsize=config.plotting.font_size + 2,
)

In [ ]:
urban_patch = mpatches.Rectangle(
    (0, 0),
    1,
    1,
    linewidth=1,
    edgecolor="none",
    facecolor="tab:grey",
    label="Urban area",
)
high_veg_patch = mpatches.Rectangle(
    (0, 0),
    1,
    1,
    linewidth=1,
    edgecolor="none",
    facecolor="tab:green",
    alpha=0.5,
    label="High vegetation",
)
low_veg_patch = mpatches.Rectangle(
    (0, 0),
    1,
    1,
    linewidth=1,
    edgecolor="none",
    facecolor="tab:olive",
    alpha=0.5,
    label="Low vegetation",
)
spacer = mpatches.Rectangle(
    (0, 0), 1, 1, linewidth=0, edgecolor="none", facecolor="none", label=""
)

In [ ]:
cmap = mcolors.ListedColormap(["tab:olive", "tab:green"])
bounds = [2, 3.5, 5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)
ax.imshow(
    vegetation_coarse_classification,
    extent=(
        vegetation_coarse_classification["x"].min(),
        vegetation_coarse_classification["x"].max(),
        vegetation_coarse_classification["y"].min(),
        vegetation_coarse_classification["y"].max(),
    ),
    interpolation="none",
    cmap=cmap,
    alpha=0.5,
    zorder=-1,
)

In [ ]:
urban_area = mpatches.Rectangle(
    (urban_offset[0], urban_offset[1]),
    1536,
    1536,
    linewidth=1,
    linestyle="--",
    edgecolor="none",
    facecolor="tab:gray",
    label="Urban area",
    zorder=3,
)
ax.add_patch(urban_area)

In [ ]:
xy_output = mpatches.Rectangle(
    (urban_offset[0] + 2 * 128, urban_offset[1] + 2 * 128),
    1536 - 4 * 128,
    1536 - 4 * 128,
    linewidth=1,
    linestyle="--",
    edgecolor="k",
    facecolor="none",
    label="Analysis area",
    zorder=3,
)
ax.add_patch(xy_output)

In [ ]:
legend = legend_ax.legend(
    handles=[
        urban_patch,
        xy_output,
        # spacer,
        high_veg_patch,
        low_veg_patch,
    ],
    loc="lower center",
    bbox_to_anchor=(0.5, 0.05),
    borderaxespad=0,
    ncol=2,
)

In [ ]:
out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
fig.savefig(out_path / "sensitivity_tests_domain.pdf", dpi=300)
fig

## Model comparison


### Domain setup

The whole domain setup will be plotted together, with domain boundaries, output boundaries, vegetation types, and LCZs.


In [ ]:
coarse_resolution = (16, 16)
coarse_gridpoints = (256, 256)
medium_coarse_resolution = (8, 8)
medium_coarse_gridpoints = (448, 448)
medium_fine_resolution = (4, 4)
medium_fine_gridpoints = (768, 768)
fine_resolution = (2, 2)
fine_gridpoints = (896, 768)

medium_coarse_offset = (448, 256)
medium_fine_offset = (896, 512)
fine_offset = (2112, 1280)
urban_offset = (
    fine_offset[0] + fine_resolution[0] * fine_gridpoints[0] - 1536,
    fine_offset[1],
)

In [ ]:
urban_offset

In [ ]:
lcz_map = xr.open_dataset(Path(config.path.data.raw) / "lcz" / "lcz_map.nc")["lcz"]
lcz_map = apply_buffer_zone_to_array(lcz_map, buffer=32)
lcz_map["x"] = lcz_map["x"] + urban_offset[0]
lcz_map["y"] = lcz_map["y"] + urban_offset[1]

In [ ]:
vegetation_map = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "INPUT"
    / "slurb_pre_default_static"
)["vegetation_type"]
vegetation_coarse_classification = xr.apply_ufunc(
    lambda x: np.where(np.isin(x, [1, 2, 3, 8, 9, 10, 11, 12, 13, 14, 15, 16]), 1, 2),
    vegetation_map,
)

Construct figure base and add axis labels.


In [ ]:
plt.ioff()
fig, axs = plt.subplots(
    2,
    1,
    height_ratios=[1, 10],
    figsize=(FIGURE_WIDTH_S, 9.5 / 2.56),
    layout="constrained",
)
ax = axs[1]
legend_ax = axs[0]
legend_ax.set_axis_off()
ax.set_aspect("equal")
ax.set_xlim((0.0, resolution[0] * gridpoints[0]))
ax.set_ylim((0.0, resolution[1] * gridpoints[1]))
ax.set_xticks(np.arange(0, 4001, 1000))
ax.set_yticks(np.arange(0, 4001, 1000))
ax.set_xlabel(r"$x$ (m)")
ax.set_ylabel(r"$y$ (m)")

Add domain bounds and surface description.


In [ ]:
ax.text(
    16,
    coarse_resolution[1] * coarse_gridpoints[1] + 16,
    "Coarse, $\Delta_i=16~\mathrm{m}$",
    va="bottom",
)
ax.text(
    medium_coarse_offset[0] + 16,
    medium_coarse_offset[1]
    + medium_coarse_resolution[1] * medium_coarse_gridpoints[1]
    + 16,
    "Medium-coarse, $\Delta_i=8~\mathrm{m}$",
    va="bottom",
)
ax.text(
    medium_fine_offset[0] + 16,
    medium_fine_offset[1] + medium_fine_resolution[1] * medium_fine_gridpoints[1] + 16,
    "Medium-fine, $\Delta_i=4~\mathrm{m}$",
    va="bottom",
)
ax.text(
    fine_offset[0] + 16,
    fine_offset[1] + fine_resolution[1] * fine_gridpoints[1] + 16,
    "Fine, $\Delta_i=2~\mathrm{m}$",
    va="bottom",
)
ax.text(
    168,
    resolution[1] * gridpoints[1] // 2,
    r"$\downarrow$ Inflow boundary $\downarrow$",
    va="center",
    ha="center",
    rotation="vertical",
    fontsize=config.plotting.font_size + 2,
)

nest_medium_coarse_bbox = mpatches.Rectangle(
    medium_coarse_offset,
    medium_coarse_resolution[0] * medium_coarse_gridpoints[0],
    medium_coarse_resolution[1] * medium_coarse_gridpoints[1],
    linewidth=1,
    edgecolor="tab:red",
    facecolor="none",
    label="Nest bounds",
)
nest_medium_fine_bbox = mpatches.Rectangle(
    medium_fine_offset,
    medium_fine_resolution[0] * medium_fine_gridpoints[0],
    medium_fine_resolution[1] * medium_fine_gridpoints[1],
    linewidth=1,
    edgecolor="tab:red",
    facecolor="none",
    label="Nest bounds",
)
nest_fine_bbox = mpatches.Rectangle(
    fine_offset,
    fine_resolution[0] * fine_gridpoints[0],
    fine_resolution[1] * fine_gridpoints[1],
    linewidth=1,
    edgecolor="tab:red",
    facecolor="none",
)
domain_bounds_patch = mpatches.Rectangle(
    (0, 0),
    1,
    1,
    linewidth=1,
    edgecolor="k",
    facecolor="none",
    label="Simulation bounds",
)
data_output_bounds_patch = mpatches.Rectangle(
    (0, 0),
    1,
    1,
    linewidth=1,
    edgecolor="k",
    facecolor="none",
    linestyle="--",
    label="Simulation bounds",
)
# nest_fine_veg = mpatches.Rectangle(fine_offset, fine_resolution[0]*fine_gridpoints[0], fine_resolution[1]*fine_gridpoints[1], linewidth=1, edgecolor="none", alpha=1.0, facecolor="#b9db7a", label="Short grass", zorder=0)
lcz_2_patch = mpatches.Rectangle(
    (0, 0), 1, 1, linewidth=1, edgecolor="none", facecolor="tab:gray", label="LCZ 2"
)
lcz_5_patch = mpatches.Rectangle(
    (0, 0), 1, 1, linewidth=1, edgecolor="none", facecolor="lightgrey", label="LCZ 5"
)
high_veg_patch = mpatches.Rectangle(
    (0, 0),
    1,
    1,
    linewidth=1,
    edgecolor="none",
    facecolor="tab:green",
    alpha=0.5,
    label="High vegetation",
)
low_veg_patch = mpatches.Rectangle(
    (0, 0),
    1,
    1,
    linewidth=1,
    edgecolor="none",
    facecolor="tab:olive",
    alpha=0.5,
    label="Low vegetation",
)
spacer = mpatches.Rectangle(
    (0, 0), 1, 1, linewidth=0, edgecolor="none", facecolor="none", label=""
)

ax.add_patch(nest_medium_coarse_bbox)
ax.add_patch(nest_medium_fine_bbox)
ax.add_patch(nest_fine_bbox)
# ax.add_patch(nest_fine_veg)

Plot LCZ grid, the corresponding colors are those which are commonly used in LCZ maps. Not super optimal but somewhat a standard.


In [ ]:
cmap = mcolors.ListedColormap(["tab:gray", "lightgray"])
bounds = [2, 3.5, 5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)
ax.imshow(
    lcz_map,
    extent=(
        lcz_map["x"].min(),
        lcz_map["x"].max(),
        lcz_map["y"].min(),
        lcz_map["y"].max(),
    ),
    interpolation="none",
    cmap=cmap,
    norm=norm,
    alpha=1.0,
)

Plot underlying vegetation.


In [ ]:
cmap = mcolors.ListedColormap(["tab:olive", "tab:green"])
# bounds = [2, 3.5, 5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)
ax.imshow(
    vegetation_coarse_classification,
    extent=(
        vegetation_coarse_classification["x"].min(),
        vegetation_coarse_classification["x"].max(),
        vegetation_coarse_classification["y"].min(),
        vegetation_coarse_classification["y"].max(),
    ),
    interpolation="none",
    cmap=cmap,
    alpha=0.5,
    zorder=-1,
)

Plot data output regions.


In [ ]:
xy_output = mpatches.Rectangle(
    (urban_offset[0] + 2 * 128, urban_offset[1] + 2 * 128),
    1536 - 4 * 128,
    1536 - 4 * 128,
    linewidth=1,
    linestyle="--",
    edgecolor="k",
    facecolor="none",
    label="Analysis area",
    zorder=3,
)
ax.add_patch(xy_output)

Add legend.


In [ ]:
legend = legend_ax.legend(
    handles=[
        nest_medium_fine_bbox,
        lcz_2_patch,
        high_veg_patch,
        xy_output,
        lcz_5_patch,
        low_veg_patch,
    ],
    loc="lower center",
    bbox_to_anchor=(0.5, 0.05),
    borderaxespad=0,
    ncol=2,
)

Plot final figure.


In [ ]:
out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
fig.savefig(out_path / "model_comparison_domains.pdf", dpi=300)
fig

### Resolved urban surface


In [ ]:
usm_run_output_path = Path(config.path.data.jobs) / "slurb_c_usm" / "OUTPUT"

In [ ]:
topo_coarse = xr.open_dataset(usm_run_output_path / "slurb_c_usm_topo_surf.000.nc")
topo_medium_coarse = xr.open_dataset(
    usm_run_output_path / "slurb_c_usm_topo_surf_N02.000.nc"
)
topo_medium_coarse["x"] = topo_medium_coarse["x"] + medium_coarse_offset[0]
topo_medium_coarse["y"] = topo_medium_coarse["y"] + medium_coarse_offset[1]

topo_medium_fine = xr.open_dataset(
    usm_run_output_path / "slurb_c_usm_topo_surf_N03.000.nc"
)
topo_medium_fine["x"] = topo_medium_fine["x"] + medium_fine_offset[0]
topo_medium_fine["y"] = topo_medium_fine["y"] + medium_fine_offset[1]

topo_fine = xr.open_dataset(usm_run_output_path / "slurb_c_usm_topo_surf_N04.000.nc")
topo_fine["x"] = topo_fine["x"] + fine_offset[0]
topo_fine["y"] = topo_fine["y"] + fine_offset[1]

In [ ]:
fig = plt.figure(figsize=(FIGURE_WIDTH_L - 1.2 / 2.56, 18 / 2.56), layout="constrained")
gs = fig.add_gridspec(
    3, 2, height_ratios=[0.05, 1, 1], width_ratios=[1, 1], wspace=0.0, hspace=0.0
)
legend_ax = fig.add_subplot(gs[0, 0])
legend_ax.set_axis_off()
cbar_ax = fig.add_subplot(gs[0, 1], aspect=1)

ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[1, 1])
ax3 = fig.add_subplot(gs[2, 0])
ax4 = fig.add_subplot(gs[2, 1])

xlims = (urban_offset[0] + 32 * 2, urban_offset[0] + 1536 - 32 * 2)
ylims = (urban_offset[1] + 32 * 2, urban_offset[1] + 1536 - 32 * 2)

lcz_vlines = urban_offset[0] + np.arange(0, 6 * 256, 256)
lcz_hlines = urban_offset[1] + np.arange(0, 6 * 256, 256)


pavement_patch = mpatches.Rectangle(
    (0, 0),
    1,
    1,
    linewidth=1,
    edgecolor="none",
    facecolor="#444444",
    alpha=1.0,
    label="Pavement",
)
veg_patch = mpatches.Rectangle(
    (0, 0),
    1,
    1,
    linewidth=1,
    edgecolor="none",
    facecolor="tab:olive",
    alpha=0.8,
    label="Vegetation",
)

legend = legend_ax.legend(
    handles=[
        pavement_patch,
        veg_patch,
    ],
    loc="lower center",
    bbox_to_anchor=(0.5, -1),
    borderaxespad=0,
    ncol=2,
)

cmap_topo = mcolors.LinearSegmentedColormap.from_list(
    name="Blues2", colors=["#00164e", "#b7c4d1"]
)

topo_coarse["buildings_2d"].plot.imshow(
    ax=ax1,
    cmap=cmap_topo,
    add_colorbar=False,
    add_labels=False,
    vmin=0.0,
    vmax=topo_fine["buildings_2d"].max(),
)
topo_medium_coarse["buildings_2d"].plot.imshow(
    ax=ax2,
    cmap=cmap_topo,
    add_colorbar=False,
    add_labels=False,
    vmin=0.0,
    vmax=topo_fine["buildings_2d"].max(),
)
topo_medium_fine["buildings_2d"].plot.imshow(
    ax=ax3,
    cmap=cmap_topo,
    add_colorbar=False,
    add_labels=False,
    vmin=0.0,
    vmax=topo_fine["buildings_2d"].max(),
)
topo = topo_fine["buildings_2d"].plot.imshow(
    ax=ax4,
    cmap=cmap_topo,
    add_colorbar=False,
    add_labels=False,
    vmin=0.0,
    vmax=topo_fine["buildings_2d"].max(),
)

cmap_pavement = mcolors.ListedColormap(["none", "#444444"])
(topo_coarse["pavement_type"] > 0).plot.imshow(
    ax=ax1, cmap=cmap_pavement, add_colorbar=False, add_labels=False
)
(topo_medium_coarse["pavement_type"] > 0).plot.imshow(
    ax=ax2, cmap=cmap_pavement, add_colorbar=False, add_labels=False
)
(topo_medium_fine["pavement_type"] > 0).plot.imshow(
    ax=ax3, cmap=cmap_pavement, add_colorbar=False, add_labels=False
)
(topo_fine["pavement_type"] > 0).plot.imshow(
    ax=ax4, cmap=cmap_pavement, add_colorbar=False, add_labels=False
)

cmap_veg = mcolors.ListedColormap(["none", "tab:olive"])
(topo_coarse["vegetation_type"] > 0).plot.imshow(
    ax=ax1, cmap=cmap_veg, add_colorbar=False, add_labels=False, alpha=0.8
)
(topo_medium_coarse["vegetation_type"] > 0).plot.imshow(
    ax=ax2, cmap=cmap_veg, add_colorbar=False, add_labels=False, alpha=0.8
)
(topo_medium_fine["vegetation_type"] > 0).plot.imshow(
    ax=ax3, cmap=cmap_veg, add_colorbar=False, add_labels=False, alpha=0.8
)
(topo_fine["vegetation_type"] > 0).plot.imshow(
    ax=ax4, cmap=cmap_veg, add_colorbar=False, add_labels=False, alpha=0.8
)

plt.colorbar(
    topo,
    cax=cbar_ax,
    orientation="horizontal",
    label="Building height (m)",
    ticks=np.arange(0, 21, 4),
)

ax1.set_title(r"Coarse, $\Delta_i=16~\mathrm{m}$")
ax2.set_title(r"Medium-coarse, $\Delta_i=8~\mathrm{m}$")
ax3.set_title(r"Medium-fine, $\Delta_i=4~\mathrm{m}$")
ax4.set_title(r"Fine, $\Delta_i=2~\mathrm{m}$")

ax1.add_artist(moffsetbox.AnchoredText("(a)", loc="upper left", pad=0.2, borderpad=0))
ax2.add_artist(moffsetbox.AnchoredText("(b)", loc="upper left", pad=0.2, borderpad=0))
ax3.add_artist(moffsetbox.AnchoredText("(c)", loc="upper left", pad=0.2, borderpad=0))
ax4.add_artist(moffsetbox.AnchoredText("(d)", loc="upper left", pad=0.2, borderpad=0))

for ax in (ax1, ax2, ax3, ax4):
    ax.set_aspect("equal")
    ax.set_xticks(np.arange(2200, 4001, 400))
    ax.set_yticks(np.arange(1400, 2601, 400))
    ax.set_xlabel("$x$ (m)")
    ax.set_ylabel("$y$ (m)")
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.vlines(
        lcz_vlines,
        ax.get_ylim()[0],
        ax.get_ylim()[1],
        linestyle="--",
        colors="white",
        alpha=0.9,
    )
    ax.hlines(
        lcz_hlines,
        ax.get_xlim()[0],
        ax.get_xlim()[1],
        linestyle="--",
        colors="white",
        alpha=0.9,
    )

for ax in (ax2, ax4):
    ax.set_ylabel("")
    ax.set_yticklabels([])

for ax in (ax1, ax2):
    ax.set_xlabel("")
    ax.set_xticklabels([])

fig.set_constrained_layout_pads(h_pad=0.05, w_pad=0.0, hspace=0.0, wspace=0.0)
out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
fig.savefig(out_path / "model_comparison_urban_form.pdf", dpi=300)
plt.show()